In [1]:
import h5py
from io import BytesIO
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import wandb
import copy
import csv

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import v2

from torchvision.models import resnet50, ResNet50_Weights, efficientnet_b3, EfficientNet_B3_Weights

In [2]:
class TestDataset(Dataset):
    def __init__(self, dataframe, hdfPath, augmentation=None):
        self.dataframe = dataframe
        self.hdf = h5py.File(hdfPath, mode="r")
        self.ids = dataframe['isic_id']
        self.augmentation = augmentation
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        itemId = self.ids[idx]
        image = Image.open(BytesIO(self.hdf[itemId][()]))
        image = np.array(image)
            
        if self.augmentation:
            image = self.augmentation(image=image)['image']
            
        return image, itemId

In [3]:
class customModel(torch.nn.Module):
    def __init__(self, pretrainedModel):
        super(customModel, self).__init__()
        self.pretrainedModel = pretrainedModel
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(p=0.25),
            torch.nn.Linear(1000, 1),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.pretrainedModel(x)
        x = self.classifier(x)
        return x

In [4]:
test_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

In [5]:
transforms_test = A.Compose([
    A.Resize(300, 300, interpolation=2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test = TestDataset(test_metadata, '/kaggle/input/isic-2024-challenge/test-image.hdf5', augmentation=transforms_test)

testLoader = DataLoader(test, batch_size=64, shuffle=False)


In [6]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead")
    device = torch.device("cpu")

GPU is available


In [7]:
pretrainedModel = efficientnet_b3()
model = customModel(pretrainedModel)
model.load_state_dict(torch.load('/kaggle/input/skin-cancer-model-training/model_params_epoch_4.pt'))
model = model.to(device)

In [8]:
with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['isic_id','target'])
    
    model.eval()
    with torch.no_grad():
        for images, cases in testLoader:
            images = images.to(device)
            preds = model(images)
            
            rows = list(zip(cases, preds.squeeze(dim=1).detach().cpu().numpy()))
            writer.writerows(rows)

In [9]:
print(f"Data has been written to submission.csv")

Data has been written to submission.csv
